In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import os

In [2]:
path_prefix = os.getcwd() + '/simulation-runs'

In [13]:
vcc = 3 # Tensão de alimentaçaõ em V
rx_current = 21.8 / 1000 # Consumo de corrente do rádio em rx em Ampére
tx_current = 19.5 / 1000 # Consumo de corrente do rádio em tx em Ampére
cpu_sleep_current = 5.1/ 1000000 # Consumo de corrente do rádio em sleep (standby) em Ampére
cpu_idle_current = 54.5 / 1000000 # Consumo de corrente do rádio em idle em Ampére

one_second_in_clock = 32768 # Um segundo em clocks do nó
interval_period = 10 # Período de envio de mensagens em segundos

In [3]:
def read_csv_in_folder(folder_path):
  dfs = []
  for filename in os.listdir(folder_path):
    # Se o arquivo é um arquivo CSV
    if filename.endswith('.csv'):
        # Construir o caminho completo para o arquivo
        file_path = os.path.join(folder_path, filename)
        
        # Ler o arquivo CSV e armazenar o DataFrame na lista
        df = pd.read_csv(file_path)
        dfs.append(df)
  return dfs

In [4]:
aloha_rdc_duty_cycles = ["5", "10", "20", "30", "40", "50", "60"]
aloha_rdc_dfs = {}

for rdc in aloha_rdc_duty_cycles:
  path = path_prefix + "/" + rdc + "-duty-cycle"
  aloha_rdc_dfs[rdc] = read_csv_in_folder(path)

In [5]:
csma_dfs = read_csv_in_folder(path_prefix + "/csma")
aloha_always_on_dfs = read_csv_in_folder(path_prefix + "/aloha-always-on")

In [6]:
def concat_dfs(dfs):
  return pd.concat(dfs, ignore_index=True)

In [7]:
# Concat dfs
aloha_rdc_dfs_concat = {}
for rdc in aloha_rdc_duty_cycles:
  aloha_rdc_dfs_concat[rdc] = concat_dfs(aloha_rdc_dfs[rdc])

csma_df_concat = pd.concat(csma_dfs, ignore_index=True)
aloha_always_on_df_concat = pd.concat(aloha_always_on_dfs, ignore_index=True)

In [15]:
# Cálculo do consumo de corrente em Ampére
def calculate_current_consumption(df):
  return (df['cpu'] * cpu_idle_current + df['transmit'] * tx_current + df['listen'] * rx_current + df['lpm'] * cpu_sleep_current) / (one_second_in_clock * interval_period)

In [22]:
#Cálculo de potência de cada linha
for rdc in aloha_rdc_duty_cycles:
  aloha_rdc_dfs_concat[rdc]['I'] = calculate_current_consumption(aloha_rdc_dfs_concat[rdc])
  aloha_rdc_dfs_concat[rdc]['P'] = aloha_rdc_dfs_concat[rdc]['I'] * vcc # Potência em W
  aloha_rdc_dfs_concat[rdc]['Eihop'] = aloha_rdc_dfs_concat[rdc]['P'] * interval_period # Energia em J

csma_df_concat['I'] = calculate_current_consumption(csma_df_concat)
csma_df_concat['P'] = csma_df_concat['I'] * vcc # Potência em W
csma_df_concat['Eihop'] = csma_df_concat['P'] * interval_period # Energia em J

aloha_always_on_df_concat['I'] = calculate_current_consumption(aloha_always_on_df_concat)
aloha_always_on_df_concat['P'] = aloha_always_on_df_concat['I'] * vcc # Potência em W
aloha_always_on_df_concat['Eihop'] = aloha_always_on_df_concat['P'] * interval_period # Energia em J

In [23]:
csma_df_concat.head()

,cpu,lpm,transmit,listen,hops,d,_R,_Nb,P0,I,P,Eihop
0,60713,922264,7812,8642,1,135,250,90,0.001064,0.001064,0.003193,0.031928
1,65511,917467,9785,8114,2,67,250,90,0.001147,0.001147,0.003442,0.034419
2,20168,307533,1716,4139,1,135,250,90,0.000386,0.000386,0.001157,0.011569
3,28207,299494,6826,6419,2,67,250,90,0.000843,0.000843,0.002528,0.025278
4,20090,307375,1594,4469,1,135,250,90,0.000400,0.000400,0.001201,0.012009


In [ ]:
# For 95% confidence interval
z = 2.228

In [ ]:
def plot_error_bar(mean, margin_of_error, title, x_label, y_label):
  plt.errorbar(0, mean, yerr=margin_of_error, fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0)
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()

In [ ]:
def calculate_mean_and_margin_of_error(data):
  mean = data.mean()
  std = data.std()
  margin_of_error = z * (std / np.sqrt(n))
  return mean, margin_of_error